# **Llama 2**

- **The Llama 2 is a collection of pretrained and fine-tuned generative text models, ranging from 7 billion to 70 billion parameters, designed for dialogue use cases.**

- **It outperforms open-source chat models on most benchmarks and is on par with popular closed-source models in human evaluations for helpfulness and safety.**

# **Llama 2 13B-chat :** https://huggingface.co/meta-llama/Llama-2-13b-chat

### we are using quantization technique --> GGML technique and library --> llama-cpp.

`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage.

#  **Quantized Models from the Hugging Face Community**

**The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.**

**There are several variations available, but the ones that interest us are based on the GGLM library. We can see the different variations that Llama-2-13B-GGML.In this case, we will use the model called [Llama-2-13B-chat-GGML].**

#**1: Install All the Required Packages**

In [1]:
!nvidia-smi

Tue Mar 19 05:51:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.5/821.5 kB 5.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 9.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 45.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 5.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 9.1 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-_hqkfcc6/overlay/local/bin
    changing mode of /tmp/pip-build-env-_hqkfcc6/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-_hqkfcc6/overlay/loc

In [3]:
# Defining the Model
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

#**2: Import All the Required Libraries**

In [4]:
from huggingface_hub import hf_hub_download

In [5]:
from llama_cpp import Llama

#**3: Download the Model**

In [6]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [7]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin'

#**4: Loading the Model**

In [8]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [9]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

#**5: Create a Prompt Template**

In [10]:
prompt = "Write a linear regression code"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

#**6: Generating the Response**

In [11]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

In [12]:
print(response)

{'id': 'cmpl-b4ad3abd-292e-4525-aae8-7c8915e8dcd4', 'object': 'text_completion', 'created': 1710827936, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression code\n\nASSISTANT:\n\nI\'d be happy to help you with that! However, I want to make sure we have the same understanding of what a "linear regression" is before we dive into writing code. Linear regression is a statistical method used to model the relationship between a dependent variable (often called y) and one or more independent variables (often called x). It assumes that the relationship can be represented by a straight line, and it tries to find the best-fitting line based on the data you provide.\n\nNow, what specific problem are you trying to solve with linear regression? D

In [14]:
response["choices"]

[{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression code\n\nASSISTANT:\n\nI\'d be happy to help you with that! However, I want to make sure we have the same understanding of what a "linear regression" is before we dive into writing code. Linear regression is a statistical method used to model the relationship between a dependent variable (often called y) and one or more independent variables (often called x). It assumes that the relationship can be represented by a straight line, and it tries to find the best-fitting line based on the data you provide.\n\nNow, what specific problem are you trying to solve with linear regression? Do you have any specific requirements for the code, such as which programming language or library should be used?',
  'index': 0,
  'logprobs': None,
  'finish_reason': 'stop'}]

In [15]:
response["choices"][0]

{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression code\n\nASSISTANT:\n\nI\'d be happy to help you with that! However, I want to make sure we have the same understanding of what a "linear regression" is before we dive into writing code. Linear regression is a statistical method used to model the relationship between a dependent variable (often called y) and one or more independent variables (often called x). It assumes that the relationship can be represented by a straight line, and it tries to find the best-fitting line based on the data you provide.\n\nNow, what specific problem are you trying to solve with linear regression? Do you have any specific requirements for the code, such as which programming language or library should be used?',
 'index': 0,
 'logprobs': None,
 'finish_reason': 'stop'}

In [13]:
print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression code

ASSISTANT:

I'd be happy to help you with that! However, I want to make sure we have the same understanding of what a "linear regression" is before we dive into writing code. Linear regression is a statistical method used to model the relationship between a dependent variable (often called y) and one or more independent variables (often called x). It assumes that the relationship can be represented by a straight line, and it tries to find the best-fitting line based on the data you provide.

Now, what specific problem are you trying to solve with linear regression? Do you have any specific requirements for the code, such as which programming language or library should be used?
